# **Introduction to Feature Columns**

**Learning objective**

1. Load a CSV file using `Pandas`
2. Create an input pipeline using `tf.data`
3. Create multiple types of feature columns

## **Introduction**

In this notebook, you classify structured data (e.g. tabular data in a CSV file) using feature columns. Feature columns serve as **a bridge to map from columns in a CSV file to features used to train an ML model**. 

## **The data set**

We will use a small data set provided by the Cleveland Clinic Foundation for Heart Disease. There are several hundred rows in the CSV. Each row describes a patient, and each column describes an attribute. We will use this information to predict whether a patient has heart disease, which in this data set is a binary classification task.

## **Import TensorFlow and other libraries**